In [1]:
from bs4 import BeautifulSoup
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; hu-HU; rv:1.7.8) Gecko/20050511 Firefox/1.0.4'}

response = requests.get("https://www.gofundme.com/s?q=medical%20bills", headers=headers)
doc = BeautifulSoup(response.text)
import re
from tqdm import tqdm

In [2]:
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

/Users/shelbyjouppi/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [70]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/shelbyjouppi/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache
/var/folders/_4/_njb58q944xdbv6xcbzbzmqh0000gn/T/ipykernel_50047/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [63]:
import random

In [44]:
df = pd.read_csv("go-fund-me-campaign-links.csv")
df = df.drop_duplicates()

In [6]:
old_links = df['0'].to_list()

In [17]:
df2 = pd.read_csv("go-fund-me-campaign-links-2.csv")
df2 = df2.drop_duplicates()

In [34]:
df2['link_list'] = df2['0'].str.split('?')

In [36]:
def clean_links(link_list):
    if len(link_list) > 1:
        return link_list[0]
    else:
        return None

In [37]:
df2['link_clean'] = df2.link_list.apply(clean_links)

In [46]:
df['link_list'] = df['0'].str.split('?')

In [47]:
df['link_clean'] = df.link_list.apply(clean_links)

In [48]:
old_links = df.link_clean.to_list()

In [54]:
new_links = df2[~df2.link_clean.isin(old_links)]['0'].to_list()

In [73]:
# scripts = []
# campaigns = []
for link in tqdm(new_links[2003:]):
    # Visit the page
    driver.get("https://www.gofundme.com"+link)
    
    # Get the java script
    initialState = driver.execute_script("return initialState")
    
    # Save the script to a list for safe keeping
    scripts.append(initialState)
    df = pd.DataFrame(scripts)
    df.to_csv("go-fund-me-scripts-2.csv", index=False)
    
    # Get campaign data from the page:
    page_data = initialState['feed']['campaign']
    
    data = {}
    
    data['name'] = page_data['fund_name']
    data['id'] = page_data['campaign_id']
    data['date'] = page_data['launch_date']
    data['city'] = page_data['location']['city']
    data['country'] = page_data['location']['country']
    data['goal'] = page_data['goal_amount']
    data['progress'] = page_data['current_amount']
    data['donations'] = page_data['donation_count']
    data['currency'] = page_data['currencycode']
    data['description'] = page_data['fund_description']
    data['deactivated'] = page_data['deactivated']
    data['url'] = link
    
    campaigns.append(data)
    df = pd.DataFrame(campaigns)
    df.to_csv("go-fund-me-campaign-data-2.csv", index=False)
    
    # Sleep
    time.sleep(random.randint(1, 4))

100%|███████████████████████████████████████| 852/852 [2:19:19<00:00,  9.81s/it]


In [69]:
len(scripts)

1746

In [74]:
df3 = pd.read_csv("go-fund-me-campaign-data-2.csv")

In [75]:
df3

,name,id,date,city,country,goal,progress,donations,currency,description,deactivated,url
0,Help a family in need,61809501,2021-12-08T11:23:08-06:00,"Blackwood, NJ",US,7500,0,0,USD,\n<div>Hi my name is Sarah and this is my uncl...,False,/f/6669q-help-a-family-in-need?qid=ef009269c90...
1,Help Sharice know we care,61737637,2021-12-05T15:41:50-06:00,"Highland Park, MI",US,10000,0,0,USD,\n<div>Hi my name is Toni Bennett and I’m doin...,False,/f/help-sharice-know-we-care?qid=ef009269c90b0...
2,Blessing for my mom,61710521,2021-12-04T09:42:49-06:00,"Winston-Salem, NC",US,3000,0,0,USD,"\n<div>Hello, my name is hallee.\n</div>\n<div...",False,/f/blessing-for-my-mom?qid=ef009269c90b07d879d...
3,Help Luke fight for SSPE,61601819,2021-11-30T13:56:28-06:00,"Surrey, BC",CA,50000,0,0,CAD,\n<div>We are raising this funds to buy his da...,False,/f/ykdcv-help-luke-fight-for-sspe?qid=ef009269...
4,This is Marceline,61598729,2021-11-30T15:35:35-06:00,"Strasburg, OH",US,100000,0,0,USD,\n<div>I gave birth to fraternal twin boys jus...,False,/f/this-is-marceline?qid=ef009269c90b07d879d3a...
...,...,...,...,...,...,...,...,...,...,...,...,...
1119,Helping hands for Julian,60299613,2021-10-03T18:29:59-05:00,"Moline, IL",US,15000,1660,19,USD,\n<div>On 8/4/21 Julian Sanchez came down with...,False,/f/helping-hands-for-julian?qid=6f0662a5aad16a...
1120,Funeral expenses for Chip Duet,61136429,2021-11-09T19:49:18-06:00,"Albright, WV",US,3000,240,5,USD,\n<div>Hi my name is Jody Duet my husband Chip...,False,/f/help-to-have-a-service-for-chip-duet?qid=6f...
1121,Funeral cost,61184917,2021-11-11T20:18:08-06:00,"Huntsville, AL",US,4500,210,4,USD,"<font color=""#333333""><br /><br /></font>\n<di...",False,/f/b6ehp-funeral-cost?qid=6f0662a5aad16a769076...
1122,Help Chris back on his feet after Covid!,60914797,2021-10-30T21:26:45-05:00,"Mazomanie, WI",US,5000,95,4,USD,\n<div>Hi my name is Gena and I am fundraising...,False,/f/help-chris-back-on-his-feet-after-covid?qid...


In [76]:
len(new_links)

2855

In [105]:
def dictionary(feed):
    return feed.to_dict()

In [107]:
df4['dict'] = df4.feed.to_dict()

In [117]:
missing_campaigns = []
for script in scripts[:1730]:
    page_data = script['feed']['campaign']
    
    data = {}
    
    data['name'] = page_data['fund_name']
    data['id'] = page_data['campaign_id']
    data['date'] = page_data['launch_date']
    data['city'] = page_data['location']['city']
    data['country'] = page_data['location']['country']
    data['goal'] = page_data['goal_amount']
    data['progress'] = page_data['current_amount']
    data['donations'] = page_data['donation_count']
    data['currency'] = page_data['currencycode']
    data['description'] = page_data['fund_description']
    data['deactivated'] = page_data['deactivated']
    data['url'] = link
    
    missing_campaigns.append(data)

In [118]:
df4 = pd.DataFrame(missing_campaigns)

In [119]:
df4

,name,id,date,city,country,goal,progress,donations,currency,description,deactivated,url
0,Von’s COVID Relief Fund,63034911,2022-02-01T15:56:43-06:00,"Concord, CA",US,18888,10100,91,USD,"\n\n<p class=""p1"" style=""line-height:normal;"">...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
1,Covid relief for Larry Forinash And family,61473321,2021-11-24T18:31:40-06:00,"Clarksburg, WV",US,5000,2490,36,USD,"\n<div>Unfortunately, on Tuesday November 16, ...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
2,Brian Fannon’s Covid Relief,62504685,2022-01-09T11:13:38-06:00,"Reno, NV",US,10000,1423,13,USD,\n<div>Hi everyone. This is Matt Erwin. It is ...,False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
3,Trisha Golds COVID relief fund,61364475,2021-11-19T19:59:28-06:00,"West Valley City, UT",US,5000,2980,37,USD,"<font color=""#333333""><span>my mother got ill ...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
4,COVID Relief and Support for the Nelson Family,61413253,2021-11-22T08:49:51-06:00,"Fuquay-Varina, NC",US,5000,1160,21,USD,"\n<div>In September 2021, the Nelson family al...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
...,...,...,...,...,...,...,...,...,...,...,...,...
1725,Need new Electric Chair.,62102959,2021-12-20T22:02:33-06:00,"Gainesville, TX",US,18000,0,0,USD,"\n<div>Hi,my name is Karen and I am trying to ...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
1726,Christmas gifts for the kids,62061361,2021-12-19T03:54:11-06:00,"Gilbert, AZ",US,3000,0,0,USD,"\n<div>This is for my daughters children, Rive...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
1727,Like many manual laborers,61992039,2021-12-15T21:30:44-06:00,"McAllen, TX",US,200000,0,0,USD,\n<div>A Laborer’s Life - Mario Gaona Ramirez ...,False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...
1728,Help raise money during brain cancer treatment,61982635,2021-12-15T14:34:35-06:00,"Williamsburg, OH",US,50000,0,0,USD,"\n<div>I’m Jessica, the past two years have be...",False,/f/q7zq8-charles039-angels?qid=6f0662a5aad16a7...


In [120]:
final_df = pd.concat([df4,df3], ignore_index=True)

In [122]:
final_df.to_csv("go-fund-me-campaigns-data-2b.csv", index=False)